# Preprocess raw EDF data to filtered FIF format

Define event names in a dictionary

In [3]:
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt

from mne.preprocessing import ICA
from pyprep.find_noisy_channels import NoisyChannels

from IPython import display

import eeglabio

Find EDF file based on subject number and load using MNE

In [4]:
parent_data_path = f"../../../George Kenefati/Dan Friedman sEEG Data/"
save_path = f"../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/" 

In [5]:
def state_and_subfile_creator(day_id):
    day_and_state = []
    for state in os.listdir(os.path.join(parent_data_path, sub_id, day_id)):
        if state.startswith(sub_state):
            state_id = sub_state;
        
    for subfile in os.listdir(os.path.join(parent_data_path,sub_id, day_id, state_id)):
        if subfile.endswith(".EDF"):
            acq_id = subfile;
            day_and_state.append(acq_id)
    day_and_state.append(state_id)
    print(day_and_state)
    return(day_and_state)



def preprocessing_function(state_id, file_name):
    print(f"{sub_id}\nreading raw file...")
    raw_fname = os.path.join(parent_data_path,sub_id, day_id, state_id, file_name)

    raw = mne.io.read_raw_edf(raw_fname, preload=True)

    display.clear_output(wait=True)

    # high level inspection
    print(raw.ch_names)
    print(len(raw.ch_names))

    # apply notch filter
    print(f"{sub_id}\napplying notch filter...")
    raw = raw.notch_filter(60., notch_widths=10)
    display.clear_output(wait=True)

    # apply bandpass filter
    print(f"{sub_id}\napplying bandpass filter...")
    if file_name.endswith("512.EDF"):
        h_freq_value = 100.
    else:
        h_freq_value = 500.
    raw = raw.filter(l_freq=1., h_freq = h_freq_value)
    display.clear_output(wait=True)

    # find bad channels automatically
    # print(f"{sub_id}\nremoving bad channels...")
    # raw_pyprep = NoisyChannels(raw, random_state=42)
    # raw_pyprep.find_all_bads(ransac=False, channel_wise=False, max_chunk_size=None)
    # raw.info['bads'] = raw_pyprep.get_bads()
    # display.clear_output(wait=True)

    # save copy of data
    save_fname_fif = f"{sub_num}_{day_list[i]}_{file_name[:-4]}-raw.fif"
    print(f"Saving processed data as '{save_fname_fif}'...")
    raw.save(save_path+save_fname_fif,
             verbose=True, overwrite=True)

    # high level inspection
    print(raw.ch_names)

In [6]:
# Subjects: [NY - 846, 853, 865, 870, 871, 872, 878, 884, 888, 893]

In [79]:
sub_num = 'NY893' # Set this at the start for each subject name
sub_state = 'Awake2'  #Set this to Asleep1 / Asleep2 / Awake1 / Awake2

In [80]:
sub_id=''; acq_id=''
for folder in os.listdir(parent_data_path):   
    if folder.endswith(sub_num):
        sub_id = folder
        print(sub_id)
        
day_list = []        
for days in os.listdir(os.path.join(parent_data_path,sub_id)):
    if days.startswith("Day"):
        day_list.append(str(days))

        
for i in range(0, len(day_list)):
    complete_info = state_and_subfile_creator(day_list[i])
    day_id = day_list[i]
    for z in range(0, 2):
        preprocessing_function(complete_info[-1], complete_info[z])

Saving processed data as 'NY893_Day 9_Awake2_512-raw.fif'...
Writing /media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Baldwin Chen/seeg-data-analysis/Preprocessing/../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/NY893_Day 9_Awake2_512-raw.fif
Closing /media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Baldwin Chen/seeg-data-analysis/Preprocessing/../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/NY893_Day 9_Awake2_512-raw.fif
[done]
['LA1', 'LA2', 'LA3', 'LA4', 'LA5', 'LA6', 'LA7', 'LA8', 'LA9', 'LA10', 'LA11', 'LA12', 'LAHC1', 'LAHC2', 'LAHC3', 'LAHC4', 'LAHC5', 'LAHC6', 'LAHC7', 'LAHC8', 'LAHC9', 'LAHC10', 'LAHC11', 'LAHC12', 'LPCH1', 'LPCH2', 'LPCH3', 'LPCH4', 'LPCH5', 'LPCH6', 'LPCH7', 'LPCH8', 'LPCH9', 'LPCH10', 'LPCH11', 'LPCH12', 'LAG1', 'LAG2', 'LAG3', 'LAG4', 'LAG5', 'LAG6', 'LAG7', 'LAG8', 'LIPH1', 'LIPH2', 'LIPH3', 'LIPH4', 'LIPH5', 'LIPH6', 'LIPH7', 'LIPH8', 'LIPH9', 'LIPH10', 'LIPH11', 'LIPH12', 'LTP1', 'LTP2', 'LTP3', 'LTP4',